In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

## 1. Data Preparation

In [ ]:
#### Merge Sensing, EMA, and student demographics Data

sensing=pd.read_csv('../Data/sensing.csv')


ema=pd.read_csv('../Data/general_ema.csv')

demo=pd.read_csv('../Data/demographics.csv')


data_merged=sensing.merge(ema,on=['uid','day'])   ### inner join

data_full=data_merged.merge(demo,on='uid') ### inner join

data_full=data_full[data_full['gender']!='both']

## 2. Raw Data Distribution

### Data Preparation

In [ ]:
# Select relevant variables
data_distribution=data_full[['uid','day','phq4_score','unlock_num_ep_0','unlock_duration_ep_0']]

# Change unit of duration from seconds to minutes
data_distribution['unlock_duration_ep_0']=data_distribution['unlock_duration_ep_0']/60

# Generate duration per unlock
data_distribution['duration_per_unlock_ep_0']=data_distribution['unlock_duration_ep_0']/data_distribution['unlock_num_ep_0']


# Compute average, min, max, and standard deviation treating NaN as zero
aggregations = {
    'unlock_num_ep_0': ['mean', 'min', 'max', 'std'],
    'unlock_duration_ep_0': ['mean', 'min', 'max', 'std'],
    'duration_per_unlock_ep_0': ['mean', 'min', 'max', 'std'],
    'phq4_score': ['mean', 'min', 'max', 'std']
}

# Group by "uid" and calculate the statistics
stats_with_zero = data_distribution.groupby('uid').agg(aggregations)

# Flatten multi-level columns
stats_with_zero.columns = ['_'.join(col) for col in stats_with_zero.columns]

# Reset index to make "uid" a column again
stats_with_zero = stats_with_zero.reset_index()

# Compute min excluding zero
def min_excluding_zero(series):
    return series[series > 0].min() if any(series > 0) else np.nan

# Group by "uid" and calculate min excluding zero
min_excluding_zero_stats = data_distribution.groupby('uid').agg({
    'unlock_num_ep_0': min_excluding_zero,
    'unlock_duration_ep_0': min_excluding_zero,
    'duration_per_unlock_ep_0': min_excluding_zero,
    'phq4_score': min_excluding_zero
}).rename(columns=lambda col: f"{col}_min_excl_zero")

# Reset index to make "uid" a column again
min_excluding_zero_stats = min_excluding_zero_stats.reset_index()

# Merge both results
final_df = pd.merge(stats_with_zero, min_excluding_zero_stats, on='uid', how='left')

final_df


### General Distribution

In [ ]:
#### Figure for Export
#### With unusual values highlighted


# List of groups and suffixes
groups = ["unlock_num_ep_0", "unlock_duration_ep_0", "duration_per_unlock_ep_0"]
suffixes = ["_mean", "_max"]

# Initialize the figure
fig, axes = plt.subplots(len(suffixes), len(groups), figsize=(20, 10), constrained_layout=True)


#### num mean

data = final_df['unlock_num_ep_0_mean'].dropna()

# Define bins with a width of 50
min_value = np.floor(data.min() / 50) * 50  # Round down to the nearest 50
max_value = np.ceil(data.max() / 50) * 50   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 50, 50)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
sns.histplot(data, kde=True, ax=axes[0, 0], bins=bin_edges, line_kws={'linewidth': 5})
axes[0, 0].set_title("(a) Mean Unlock Number", fontsize=30, fontweight='bold', pad=20)

# Set custom bin edges as x-ticks
axes[0, 0].set_xticks(bin_edges)
axes[0, 0].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Convert edges to integers

# Set y-axis label
axes[0, 0].set_ylabel("Number of Students", fontsize=30)

# Increase y-tick font size
axes[0, 0].tick_params(axis='y', labelsize=30)

# Set x-axis label
axes[0, 0].set_xlabel("Unit: Times", fontsize=30)





#### duration mean


# Drop NaN values and convert data to hours
data = final_df['unlock_duration_ep_0_mean'].dropna()
data = data / 60  # Convert to hours

# Define bins with a width of 1
min_value = np.floor(data.min() / 1) * 1  # Round down to the nearest integer
max_value = np.ceil(data.max() / 1) * 1   # Round up to the nearest integer
bin_edges = np.arange(min_value, max_value + 1, 1)  # Create bins with width of 1

# Plot histogram using seaborn with custom bins
sns.histplot(data, kde=True, ax=axes[0, 1], bins=bin_edges, line_kws={'linewidth': 5})
axes[0, 1].set_title("(b) Mean Unlock Duration", fontsize=30,fontweight='bold', pad=20)  # Increased title font size

# Set bin edges as x-ticks
axes[0, 1].set_xticks(bin_edges)
axes[0, 1].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Increased x-tick font size

# Set y-axis label
axes[0, 1].set_ylabel("")  # No label for y-axis as per original code

# Increase y-tick font size
axes[0, 1].tick_params(axis='y', labelsize=30)

# Set x-axis label
axes[0, 1].set_xlabel('Unit: Hours', fontsize=30)





#### duration/unlock mean
data = final_df['duration_per_unlock_ep_0_mean'].dropna()

# Define bins with a width of 1
min_value = np.floor(data.min() / 2) * 2  # Round down to the nearest integer
max_value = np.ceil(data.max() / 2) * 2   # Round up to the nearest integer
bin_edges = np.arange(min_value, max_value + 2, 2)  # Create bins with width of 1

# Plot histogram using seaborn with custom bins
sns.histplot(data, kde=True, ax=axes[0, 2], bins=bin_edges,line_kws={'linewidth': 5})
axes[0, 2].set_title(f"(c) Mean Duration per Unlock", fontsize=30, fontweight='bold', pad=20)  # Increased title font size

# Set rounded bin edges as x-ticks
axes[0, 2].set_xticks(bin_edges)
axes[0, 2].set_xticklabels([f"{int(edge)}" if edge.is_integer() else f"{edge:.1f}" for edge in bin_edges], rotation=45, fontsize=30)
#axes[0, 2].set_xticklabels(bin_edges_rounded, rotation=45, fontsize=20)  # Increased x-tick font size

# Set y-axis label
axes[0, 2].set_ylabel("")

# Increase y-tick font size
axes[0, 2].tick_params(axis='y', labelsize=30)

axes[0, 2].set_xlabel('Unit: Minutes', fontsize=30)




#### num max


# Data preparation

data = final_df['unlock_num_ep_0_max'].dropna()

# Define bins with a width of 50
min_value = np.floor(data.min() / 100) * 100  # Round down to the nearest 50
max_value = np.ceil(data.max() / 100) * 100   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 100, 100)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
hist = sns.histplot(data, kde=True, ax=axes[1, 0], bins=bin_edges, line_kws={'linewidth': 5})

# Highlight bars whose bin edges contain zero
for patch, left_edge in zip(hist.patches, bin_edges[:-1]):
    if left_edge <= 0 < left_edge + 50:  # Check if the bin includes zero
        patch.set_facecolor('red')  # Highlight in red

# Format x-tick labels to display bin edges
bin_edges_rounded = bin_edges.astype(int)
axes[1, 0].set_xticks(bin_edges)
axes[1, 0].set_xticklabels(bin_edges_rounded, rotation=45, fontsize=30)

# Add title and labels
axes[1, 0].set_title("(d) Max Unlock Number", fontsize=30, fontweight='bold', pad=20)
axes[1, 0].set_ylabel("Number of Students", fontsize=30)
axes[1, 0].set_xlabel('Unit: Times', fontsize=30)

# Adjust y-axis tick font size
axes[1, 0].tick_params(axis='y', labelsize=25)








#### duration max

# Drop NaN values from the data and convert to hours
data = final_df['unlock_duration_ep_0_max'].dropna()
data = data / 60  # Convert minutes to hours

# Define bins with a width of 50
min_value = np.floor(data.min() / 2) * 2  # Round down to the nearest 50
max_value = np.ceil(data.max() / 2) * 2  # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 2, 2)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
hist = sns.histplot(data, kde=True, ax=axes[1, 1], bins=bin_edges, line_kws={'linewidth': 5})

# Highlight bars whose x-axis values are zero or above 1000
for patch, left_edge in zip(hist.patches, bin_edges[:-1]):
    if left_edge == 0:  # Highlight zero bins
        patch.set_facecolor('red')
    elif left_edge >= 16:  # Highlight bins above or equal to 1000
        patch.set_facecolor('red')

# Add title and labels
axes[1, 1].set_title(f"(e) Max Unlock Duration", fontsize=30, fontweight='bold', pad=20)

# Set x-ticks and labels
axes[1, 1].set_xticks(bin_edges)
axes[1, 1].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=25)

# Set axis labels and font sizes
axes[1, 1].set_ylabel("")
axes[1, 1].set_xlabel('Unit: Hours', fontsize=30)
axes[1, 1].tick_params(axis='y', labelsize=30)






#### duration/unlock mean
data = final_df['duration_per_unlock_ep_0_max'].dropna()

data = data / 60  # Convert minutes to hours

# Define bins with a width of 50
min_value = np.floor(data.min() / 1) * 1  # Round down to the nearest 50
max_value = np.ceil(data.max() / 1) * 1  # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 1, 1)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
sns.histplot(data, kde=True, ax=axes[1, 2], bins=bin_edges,line_kws={'linewidth': 5})
axes[1, 2].set_title(f"(f) Max Duration per Unlock", fontsize=30, fontweight='bold', pad=20)  # Increased title font size

# Set rounded bin edges as x-ticks
axes[1, 2].set_xticks(bin_edges)
axes[1, 2].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Increased x-tick font size

# Set y-axis label
axes[1, 2].set_ylabel("")

# Increase y-tick font size
axes[1, 2].tick_params(axis='y', labelsize=30)

axes[1, 2].set_xlabel('Unit: Hours', fontsize=30)


# Iterate through each subplot and add bar heights
for ax_row in axes:
    for ax in ax_row:
        for bar in ax.patches:
            height = bar.get_height()  # Get the height of the bar
            if height > 0:  # Only annotate bars with height greater than 0
                ax.text(
                    bar.get_x() + bar.get_width() / 2,  # X-coordinate at the center of the bar
                    height,  # Y-coordinate just above the bar
                    f"{int(height)}",  # Convert height to integer for display
                    ha='center', va='bottom', fontsize=27, color='black'  # Formatting
                )

# Export the figure as a PDF
output_path = "../Figures/distribution_highlight.pdf"  # Change this to your desired file path
fig.savefig(output_path, format='pdf', bbox_inches='tight')

plt.show()

### Distribution by Gender

In [ ]:
final_df_gender=final_df.merge(demo,on='uid')


#### Figure for Export
#### With unusual values highlighted

# Define colors for genders
colors = {'M': 'blue', 'F': 'red'}
# List of groups and suffixes
groups = ["unlock_num_ep_0", "unlock_duration_ep_0", "duration_per_unlock_ep_0"]
suffixes = ["_mean", "_max"]

# Initialize the figure
fig, axes = plt.subplots(len(suffixes), len(groups), figsize=(20, 10), constrained_layout=True)


#### num mean

# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['unlock_num_ep_0_mean'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['unlock_num_ep_0_mean'].dropna()

# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 50) * 50  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 50) * 50   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 50, 50)  # Create bins with width of 50

# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[0, 0], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[0, 0], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

# Add title and labels
axes[0, 0].set_title("(a) Mean Unlock Number", fontsize=30, fontweight='bold')
axes[0, 0].set_xlabel("Unit: Times", fontsize=30)
axes[0, 0].set_ylabel("Number of Students", fontsize=30)

# Set custom bin edges as x-ticks
axes[0, 0].set_xticks(bin_edges)
axes[0, 0].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Convert edges to integers

# Increase y-tick font size
axes[0, 0].tick_params(axis='y', labelsize=30)

# Add legend
axes[0, 0].legend(title='Gender', fontsize=20, title_fontsize=20)








#### duration mean


# Drop NaN values and convert data to hours
# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['unlock_duration_ep_0_mean'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['unlock_duration_ep_0_mean'].dropna()

data_male=data_male/60
data_female=data_female/60

# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 1) * 1  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 1) * 1   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 1, 1)  # Create bins with width of 50

# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[0, 1], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[0, 1], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

# Add title and labels
axes[0, 1].set_title("(b) Mean Unlock Duration", fontsize=30, fontweight='bold')
axes[0, 1].set_xlabel("Unit: Hours", fontsize=30)
axes[0, 1].set_ylabel("Number of Students", fontsize=30)

# Set custom bin edges as x-ticks
axes[0, 1].set_xticks(bin_edges)
axes[0, 1].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Convert edges to integers

# Increase y-tick font size
axes[0, 1].tick_params(axis='y', labelsize=30)

# Add legend
axes[0, 1].legend(title='Gender', fontsize=20, title_fontsize=20)





#### duration/unlock mean

# Drop NaN values and convert data to hours
# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['duration_per_unlock_ep_0_mean'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['duration_per_unlock_ep_0_mean'].dropna()


# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 2) * 2  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 2) * 2   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 2, 2)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[0, 2], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[0, 2], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

axes[0, 2].set_title(f"(c) Mean Duration per Unlock", fontsize=30, fontweight='bold')  # Increased title font size

# Set rounded bin edges as x-ticks
axes[0, 2].set_xticks(bin_edges)
axes[0, 2].set_xticklabels([f"{int(edge)}" if edge.is_integer() else f"{edge:.1f}" for edge in bin_edges], rotation=45, fontsize=30)
#axes[0, 2].set_xticklabels(bin_edges_rounded, rotation=45, fontsize=20)  # Increased x-tick font size

# Set y-axis label
axes[0, 2].set_ylabel("")

# Increase y-tick font size
axes[0, 2].tick_params(axis='y', labelsize=30)

axes[0, 2].set_xlabel('Unit: Minutes', fontsize=30)

# Add legend
axes[0, 2].legend(title='Gender', fontsize=20, title_fontsize=20)



#### num max

# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['unlock_num_ep_0_max'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['unlock_num_ep_0_max'].dropna()

# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 100) * 100  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 100) * 100   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 100, 100)  # Create bins with width of 50

# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[1, 0], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[1, 0], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

# Add title and labels
axes[1, 0].set_title("(d) Max Unlock Number", fontsize=30, fontweight='bold')
axes[1, 0].set_xlabel("Unit: Times", fontsize=30)
axes[1, 0].set_ylabel("Number of Students", fontsize=30)

# Set custom bin edges as x-ticks
axes[1, 0].set_xticks(bin_edges)
axes[1, 0].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Convert edges to integers

# Increase y-tick font size
axes[1, 0].tick_params(axis='y', labelsize=30)

# Add legend
axes[1, 0].legend(title='Gender', fontsize=20, title_fontsize=20)






#### duration max

# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['unlock_duration_ep_0_max'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['unlock_duration_ep_0_max'].dropna()

data_male=data_male/60
data_female=data_female/60

# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 2) * 2  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 2) * 2   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 2, 2)  # Create bins with width of 50

# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[1, 1], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[1, 1], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

# Add title and labels
axes[1, 1].set_title("(e) Max Unlock Duration", fontsize=30, fontweight='bold')
axes[1, 1].set_xlabel("Unit: Hours", fontsize=30)
axes[1, 1].set_ylabel("Number of Students", fontsize=30)

# Set custom bin edges as x-ticks
axes[1, 1].set_xticks(bin_edges)
axes[1, 1].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=25)  # Convert edges to integers

# Increase y-tick font size
axes[1, 1].tick_params(axis='y', labelsize=30)

# Add legend
axes[1, 1].legend(title='Gender', fontsize=20, title_fontsize=20)








#### duration/unlock mean
# Filter data by gender
data_male = final_df_gender[final_df_gender['gender'] == 'M']['duration_per_unlock_ep_0_max'].dropna()
data_female = final_df_gender[final_df_gender['gender'] == 'F']['duration_per_unlock_ep_0_max'].dropna()

data_male=data_male/60
data_female=data_female/60

# Define bins with a width of 50
min_value = np.floor(min(data_male.min(), data_female.min()) / 1) * 1  # Round down to the nearest 50
max_value = np.ceil(max(data_male.max(), data_female.max()) / 1) * 1   # Round up to the nearest 50
bin_edges = np.arange(min_value, max_value + 1, 1)  # Create bins with width of 50

# Plot histogram using seaborn with custom bins
# Plot histograms for male and female data
sns.histplot(data_male, kde=True, bins=bin_edges, ax=axes[1, 2], color='blue', label='Male', line_kws={'linewidth': 4}, alpha=0.4)
sns.histplot(data_female, kde=True, bins=bin_edges, ax=axes[1, 2], color='red', label='Female', line_kws={'linewidth': 4},alpha=0.4)

axes[1, 2].set_title(f"(f) Max Duration per Unlock", fontsize=30, fontweight='bold')  # Increased title font size

# Set rounded bin edges as x-ticks
axes[1, 2].set_xticks(bin_edges)
axes[1, 2].set_xticklabels(bin_edges.astype(int), rotation=45, fontsize=30)  # Increased x-tick font size

# Set y-axis label
axes[1, 2].set_ylabel("")

# Increase y-tick font size
axes[1, 2].tick_params(axis='y', labelsize=30)

axes[1, 2].set_xlabel('Unit: Hours', fontsize=30)

# Add legend
axes[1, 2].legend(title='Gender', fontsize=20, title_fontsize=20)




# Export the figure as a PDF
output_path = "../Figures/distribution_by_gender.pdf"  # Change this to your desired file path
fig.savefig(output_path, format='pdf', bbox_inches='tight')

plt.show()


## 3. Pearson Correlation Analysis

### Data after exclusion

In [ ]:
## Convert hours to minutes
data_full['unlock_duration_ep_0']=data_full['unlock_duration_ep_0']/60

## exclude rows whose duration is over 16 hours
data_new=data_full[data_full['unlock_duration_ep_0']<=1000]

In [ ]:
data_distribution_new=data_new[['uid','day','phq4_score','unlock_num_ep_0','unlock_duration_ep_0']]

#data_distribution['unlock_duration_ep_0']=data_distribution['unlock_duration_ep_0']/60

data_distribution_new['duration_per_unlock_ep_0']=data_distribution_new['unlock_duration_ep_0']/data_distribution_new['unlock_num_ep_0']


# Assume df is your dataframe
# Replace NaN with zero for calculation
#df_filled = data_distribution.fillna(0)
#df_filled['duration_per_unlock_ep_0']=df_filled['unlock_duration_ep_0']/df_filled['unlock_num_ep_0']

# Compute average, min, max, and standard deviation treating NaN as zero
aggregations = {
    'unlock_num_ep_0': ['mean', 'min', 'max', 'std'],
    'unlock_duration_ep_0': ['mean', 'min', 'max', 'std'],
    'duration_per_unlock_ep_0': ['mean', 'min', 'max', 'std'],
    'phq4_score': ['mean', 'min', 'max', 'std']
}

# Group by "uid" and calculate the statistics
stats_with_zero = data_distribution_new.groupby('uid').agg(aggregations)

# Flatten multi-level columns
stats_with_zero.columns = ['_'.join(col) for col in stats_with_zero.columns]

# Reset index to make "uid" a column again
stats_with_zero = stats_with_zero.reset_index()

# Compute min excluding zero
def min_excluding_zero(series):
    return series[series > 0].min() if any(series > 0) else np.nan

# Group by "uid" and calculate min excluding zero
min_excluding_zero_stats = data_distribution_new.groupby('uid').agg({
    'unlock_num_ep_0': min_excluding_zero,
    'unlock_duration_ep_0': min_excluding_zero,
    'duration_per_unlock_ep_0': min_excluding_zero,
    'phq4_score': min_excluding_zero
}).rename(columns=lambda col: f"{col}_min_excl_zero")

# Reset index to make "uid" a column again
min_excluding_zero_stats = min_excluding_zero_stats.reset_index()

# Merge both results
final_df_new = pd.merge(stats_with_zero, min_excluding_zero_stats, on='uid', how='left')

final_df_new




In [ ]:
##
data_new=data_new.merge(final_df_new,on='uid',how='left')

data_new=data_new[(data_new['unlock_num_ep_0_std']>0) & (data_new['unlock_duration_ep_0_std']>0) & (data_new['phq4_score_std']>0)]

data_new=data_new[(data_new['unlock_num_ep_0_max']>0) & (data_new['unlock_duration_ep_0_max']>0)]


### Data Preprocessing

In [ ]:
df=data_new

df['day'] = pd.to_datetime(df['day'], format='%Y%m%d')

df=df.fillna(0).copy()

## Generate Moving Average

merged_df1= df.sort_values(by=['uid', 'day'])

window_size = 14
newdf=merged_df1
newdf



In [ ]:


# Define a function to calculate rolling metrics for all columns except 'UID' and 'day'
def calculate_rolling_metrics(group):
    col_to_avg=[
     'unlock_duration_ep_0',
     'unlock_num_ep_0',

     'unlock_duration_ep_1',
     'unlock_num_ep_1',

     'unlock_duration_ep_2',
     'unlock_num_ep_2',

     'unlock_duration_ep_3',
     'unlock_num_ep_3',

     'loc_self_dorm_unlock_duration',
     'loc_self_dorm_unlock_num',

     'loc_study_unlock_duration',
     'loc_study_unlock_num',

     'loc_social_unlock_duration',
     'loc_social_unlock_num',

     'loc_food_unlock_duration',
     'loc_food_unlock_num',

     'loc_home_unlock_duration',
     'loc_home_unlock_num',
     ]

    group = group.sort_values('day')  # Ensure sorting within each group

    # Apply rolling calculations for each numeric column
    for col in group.columns:
        if col not in ['uid', 'day'] and col in col_to_avg:  # Skip non-numeric columns
            group[f'{col}_2w_sum'] = group.rolling('14D', on='day')[col].sum()
            group[f'{col}_2w_med'] = group.rolling('14D', on='day')[col].median()
            group[f'{col}_2w_avg'] = group.rolling('14D', on='day')[col].mean()

    return group

# # Group by UID and apply the rolling function
# newdf_mod = (
#     newdf_mod
#     .groupby('uid', group_keys=False)
#     .apply(calculate_rolling_metrics)
# )

# newdf_mod

newdf = (
    newdf
    .groupby('uid', group_keys=False)
    .apply(calculate_rolling_metrics)
)

newdf



In [ ]:
newdf_everything=newdf[['uid',
 'day', 'gender',
 'unlock_duration_ep_0',
 'unlock_num_ep_0',
 'unlock_duration_ep_1',
 'unlock_num_ep_1',
 'unlock_duration_ep_2',
 'unlock_num_ep_2',
 'unlock_duration_ep_3',
 'unlock_num_ep_3',
 'loc_self_dorm_unlock_duration',
 'loc_self_dorm_unlock_num',
 'loc_study_unlock_duration',
 'loc_study_unlock_num',
 'loc_social_unlock_duration',
 'loc_social_unlock_num',
 'loc_food_unlock_duration',
 'loc_food_unlock_num',
 'loc_home_unlock_duration',
 'loc_home_unlock_num',
 'unlock_duration_ep_0_2w_avg', 'unlock_duration_ep_0_2w_med',
       'unlock_duration_ep_0_2w_sum', 'unlock_num_ep_0_2w_avg',
       'unlock_num_ep_0_2w_med', 'unlock_num_ep_0_2w_sum',
       'unlock_duration_ep_1_2w_avg', 'unlock_duration_ep_1_2w_med',
       'unlock_duration_ep_1_2w_sum', 'unlock_num_ep_1_2w_avg',
       'unlock_num_ep_1_2w_med', 'unlock_num_ep_1_2w_sum',
       'unlock_duration_ep_2_2w_avg', 'unlock_duration_ep_2_2w_med',
       'unlock_duration_ep_2_2w_sum', 'unlock_num_ep_2_2w_avg',
       'unlock_num_ep_2_2w_med', 'unlock_num_ep_2_2w_sum',
       'unlock_duration_ep_3_2w_avg', 'unlock_duration_ep_3_2w_med',
       'unlock_duration_ep_3_2w_sum', 'unlock_num_ep_3_2w_avg',
       'unlock_num_ep_3_2w_med', 'unlock_num_ep_3_2w_sum',
       'loc_self_dorm_unlock_duration_2w_avg',
       'loc_self_dorm_unlock_duration_2w_med',
       'loc_self_dorm_unlock_duration_2w_sum',
       'loc_self_dorm_unlock_num_2w_avg', 'loc_self_dorm_unlock_num_2w_med',
       'loc_self_dorm_unlock_num_2w_sum', 'loc_study_unlock_duration_2w_avg',
       'loc_study_unlock_duration_2w_med',
       'loc_study_unlock_duration_2w_sum', 'loc_study_unlock_num_2w_avg',
       'loc_study_unlock_num_2w_med', 'loc_study_unlock_num_2w_sum',
       'loc_social_unlock_duration_2w_avg',
       'loc_social_unlock_duration_2w_med',
       'loc_social_unlock_duration_2w_sum', 'loc_social_unlock_num_2w_avg',
       'loc_social_unlock_num_2w_med', 'loc_social_unlock_num_2w_sum',
       'loc_food_unlock_duration_2w_avg', 'loc_food_unlock_duration_2w_med',
       'loc_food_unlock_duration_2w_sum', 'loc_food_unlock_num_2w_avg',
       'loc_food_unlock_num_2w_med', 'loc_food_unlock_num_2w_sum',
       'loc_home_unlock_duration_2w_avg', 'loc_home_unlock_duration_2w_med',
       'loc_home_unlock_duration_2w_sum', 'loc_home_unlock_num_2w_avg',
       'loc_home_unlock_num_2w_med', 'loc_home_unlock_num_2w_sum','phq4_score']]

newdf_everything.head(20)

In [ ]:
newdf_everything=newdf_everything.dropna(subset=[
       'phq4_score']).copy()

In [ ]:
newdf_everything.to_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv', index=False)

### Pearson Correlation

In [ ]:
data_corr=pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')

demo=pd.read_csv('../Data/Demographics/demographics.csv')

data_corr=data_corr.merge(demo,on='uid')

## Generate Duration per Unlock
data_corr['duration_per_unlock_ep_0_2w_avg']=data_corr['unlock_duration_ep_0_2w_avg']/data_corr['unlock_num_ep_0_2w_avg']

## Change Units
data_corr['duration_per_unlock_ep_0_2w_avg']=data_corr['duration_per_unlock_ep_0_2w_avg']/60

data_corr['unlock_duration_ep_0_2w_avg']=data_corr['unlock_duration_ep_0_2w_avg']/3600



#### Overall and Gender Specific

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

# Function to calculate correlation and p-value with cleaning
def calculate_corr_and_pval(data, var):
    clean_data = data[['phq4_score', var]].dropna()  # Drop rows with NaNs
    clean_data = clean_data[np.isfinite(clean_data['phq4_score']) & np.isfinite(clean_data[var])]  # Drop rows with infs
    if len(clean_data) > 1:  # Ensure there are enough data points
        corr, pval = pearsonr(clean_data['phq4_score'], clean_data[var])
    else:
        corr, pval = np.nan, np.nan  # Return NaN if not enough data
    return corr, pval

# Filter the data for male and female
data_male = data_corr[data_corr['gender'] == 'M']
data_female = data_corr[data_corr['gender'] == 'F']

# Define the variables for correlation
variables = ['unlock_duration_ep_0_2w_avg', 'unlock_num_ep_0_2w_avg', 'duration_per_unlock_ep_0_2w_avg']

# Initialize dictionaries for correlations and p-values
results = {'Overall Correlation': [], 'Overall P-value': [],
           'Male Correlation': [], 'Male P-value': [],
           'Female Correlation': [], 'Female P-value': []}

# Loop through variables and calculate correlations and p-values
for var in variables:
    # Overall
    corr, pval = calculate_corr_and_pval(data_corr, var)
    results['Overall Correlation'].append(corr)
    results['Overall P-value'].append(pval)

    # Male
    corr, pval = calculate_corr_and_pval(data_male, var)
    results['Male Correlation'].append(corr)
    results['Male P-value'].append(pval)

    # Female
    corr, pval = calculate_corr_and_pval(data_female, var)
    results['Female Correlation'].append(corr)
    results['Female P-value'].append(pval)

# Create a DataFrame to display the results
correlation_table = pd.DataFrame(results, index=['Unlock Duration', 'Unlock Number', 'Duration per Unlock'])

# Export the table to a CSV file
output_file = '../Tables/correlation_table_with_pvalues.csv'
correlation_table.to_csv(output_file)

# Display the table
print(correlation_table)


#### Location

In [ ]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np

locations = ['study', 'social', 'food', 'home', 'self_dorm']

for loc in locations:
    # Create duration per unlock feature
    data_corr[f'loc_{loc}_duration_per_unlock_2w_avg'] = data_corr[f'loc_{loc}_unlock_duration_2w_avg'] / data_corr[f'loc_{loc}_unlock_num_2w_avg']

    # Filter the data for male and female
    data_male = data_corr[data_corr['gender'] == 'M']
    data_female = data_corr[data_corr['gender'] == 'F']

    # Define the variables for correlation
    variables = [f'loc_{loc}_duration_per_unlock_2w_avg', f'loc_{loc}_unlock_duration_2w_avg', f'loc_{loc}_unlock_num_2w_avg']

    # Initialize results dictionary
    results = {'Overall Correlation': [], 'Overall P-value': [],
               'Male Correlation': [], 'Male P-value': [],
               'Female Correlation': [], 'Female P-value': []}

    # Loop through variables and calculate correlations and p-values
    for var in variables:
        # Overall
        overall_corr, overall_pval = pearsonr(data_corr.dropna(subset=['phq4_score', var])['phq4_score'],
                                              data_corr.dropna(subset=['phq4_score', var])[var])
        results['Overall Correlation'].append(overall_corr)
        results['Overall P-value'].append(overall_pval)

        # Male
        male_corr, male_pval = pearsonr(data_male.dropna(subset=['phq4_score', var])['phq4_score'],
                                        data_male.dropna(subset=['phq4_score', var])[var])
        results['Male Correlation'].append(male_corr)
        results['Male P-value'].append(male_pval)

        # Female
        female_corr, female_pval = pearsonr(data_female.dropna(subset=['phq4_score', var])['phq4_score'],
                                            data_female.dropna(subset=['phq4_score', var])[var])
        results['Female Correlation'].append(female_corr)
        results['Female P-value'].append(female_pval)

    # Create a DataFrame to display the results
    correlation_table = pd.DataFrame(results, index=['Duration per Unlock', 'Unlock Duration', 'Unlock Number'])

    # Export the table to a CSV file
    output_file = f'../Tables/correlation_table_{loc}.csv'
    correlation_table.to_csv(output_file)

    # Display the table
    print(f"Correlation Table for {loc.capitalize()}:")
    print(correlation_table)


## 4. Logistic Regression Model

In [ ]:
df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'ep_0' in c:
        if 'avg' in c:
            features.append(c)

features

In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Normal': log_reg.coef_[0],
    'Coefficient_Mild': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Gender Specific - Male

In [ ]:
df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

filtered_df = df[df['gender'] == 'M']

In [ ]:
filtered_df.columns
features = []

for c in list(filtered_df.columns):
    if 'ep_0' in c:
        if 'avg' in c:
            features.append(c)


target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
filtered_df[target] = label_encoder.fit_transform(filtered_df[target])

In [ ]:
# Split the data into training and testing sets
X = filtered_df[features]
y = filtered_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Gender - Female

In [ ]:
df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

In [ ]:
filtered_df = df[df['gender'] == 'F']
filtered_df

#### Logistic

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(filtered_df.columns):
    if 'ep_0' in c:
        if 'avg' in c:
            features.append(c)

features


In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
filtered_df[target] = label_encoder.fit_transform(filtered_df[target])

In [ ]:
# Split the data into training and testing sets
X = filtered_df[features]
y = filtered_df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Location -food

#### read data

In [ ]:
# df= pd.read_csv('/Users/meghnaroy/Library/CloudStorage/OneDrive-purdue.edu/Research/MentalHealth/MOBISOFT/Formatted_Not_Normalized_Everything.csv')
# df

df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


In [ ]:
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

#### Logistic


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'food' in c:
        if 'avg' in c:
            features.append(c)



In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### location - Social

#### read data

In [ ]:

df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


In [ ]:
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

#### Logistic


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'social' in c:
        if 'avg' in c:
            features.append(c)

features


In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Location - home

#### read data

In [ ]:


df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')

In [ ]:
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

#### Logistic


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'home' in c:
        if 'avg' in c:
            features.append(c)

features


In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Location -DORM

#### read data

In [ ]:

df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


In [ ]:
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

#### Logistic


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'dorm' in c:
        if 'avg' in c:
            features.append(c)

features


In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature

coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())


### Location - study

#### read data

In [ ]:


df= pd.read_csv('../Data/Formatted_Not_Normalized_Everything_Mobisoft.csv')


In [ ]:
# First, categorize the PHQ4 scores into the specified categories
def categorize_phq4(score):
    if 0 <= score <= 2:
        return 'Normal'
    elif 3 <= score <= 5:
        return 'Mild'
    elif 6 <= score <= 8:
        return 'Moderate'
    elif 9 <= score <= 12:
        return 'Severe'
    else:
        return 'Unknown'

# Add a new column for PHQ4 category
df['phq4_category'] = df['phq4_score'].apply(categorize_phq4)

#### Logistic


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, classification_report
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dense, Flatten, Dropout
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [ ]:
features=[]

for c in list(df.columns):
    if 'study' in c:
        if 'avg' in c:
            features.append(c)

features


In [ ]:
target = 'phq4_category'

# Encode the target variable (phq4_category) into numerical labels
label_encoder = LabelEncoder()
df[target] = label_encoder.fit_transform(df[target])

In [ ]:
# Split the data into training and testing sets
X = df[features]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Perform multi-class logistic regression
log_reg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000 , C=0.1)
log_reg.fit(X_train_scaled, y_train)

In [ ]:
# Step 3: Find the coefficients of each feature
coefficients = pd.DataFrame({
    'Feature': features,
    'Coefficient_Mild': log_reg.coef_[0],
    'Coefficient_Normal': log_reg.coef_[1],
    'Coefficient_Moderate': log_reg.coef_[2],
    'Coefficient_Severe': log_reg.coef_[3],
})
coefficients

In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Add an intercept to the feature matrix
X_train_with_intercept = sm.add_constant(X_train_scaled)

# Fit the multinomial logistic regression model using statsmodels
mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept)

# Custom fit function with L2 regularization
def fit_with_regularization(model, reg_param=0.1, maxiter=1000):
    """
    Fits the MNLogit model with L2 regularization.

    Parameters:
        model: statsmodels.MNLogit instance
        reg_param: Regularization parameter (similar to 1/C in sklearn)
        maxiter: Maximum number of iterations

    Returns:
        Result object similar to statsmodels fit
    """
    # Initialize starting parameters as zeros
    start_params = np.zeros(model.exog.shape[1] * (model.J - 1))

    def regularized_loglike(params):
        loglike = model.loglike(params)  # Log-likelihood from MNLogit
        penalty = reg_param * np.sum(params**2)  # L2 penalty
        return -(loglike - penalty)  # Negative log-likelihood for minimization

    def regularized_score(params):
        score = model.score(params)  # Score (gradient) from MNLogit
        penalty_gradient = 2 * reg_param * params  # Gradient of L2 penalty
        return -(score - penalty_gradient)

    # Use 'lbfgs' solver for optimization
    result = minimize(
        fun=regularized_loglike,
        x0=start_params,
        jac=regularized_score,
        method='L-BFGS-B',
        options={'maxiter': maxiter, 'disp': True},
    )

    # Validate convergence and return parameters
    if not result.success:
        raise ValueError("Optimization did not converge: " + result.message)

    # Wrap the results for compatibility with statsmodels API
    return model.fit(start_params=result.x, maxiter=maxiter, method='lbfgs', disp=False)

# Fit the model with L2 regularization
result = fit_with_regularization(mnlogit_model, reg_param=0.1, maxiter=1000)

# Display the summary with p-values
print(result.summary())
